In [2]:
# ======================================
# 1. Import Libraries
# ======================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam


ImportError: Traceback (most recent call last):
  File "c:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 73, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed while importing _pywrap_tensorflow_internal: A dynamic link library (DLL) initialization routine failed.


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.

In [1]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# ======================================
# 2. Load Dataset
# ======================================
data = pd.read_csv("./data/normalized_features.csv")

X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


# ======================================
# 3. DNN Model Function
# ======================================
def create_dnn(input_dim):
    model = Sequential([
        Dense(64, activation='relu', input_shape=(input_dim,)),
        Dropout(0.3),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model


# ======================================
# 4. 1️⃣ Simple DNN
# ======================================
simple_dnn = create_dnn(X_train.shape[1])

history_simple = simple_dnn.fit(
    X_train, y_train,
    epochs=30,
    batch_size=32,
    validation_split=0.2,
    verbose=0
)

y_pred_simple = (simple_dnn.predict(X_test) > 0.5).astype(int)
acc_simple = accuracy_score(y_test, y_pred_simple)


# ======================================
# 5. 2️⃣ Feature Selection + DNN
# ======================================
selector = SelectKBest(score_func=f_classif, k=10)
X_train_fs = selector.fit_transform(X_train, y_train)
X_test_fs = selector.transform(X_test)

fs_dnn = create_dnn(X_train_fs.shape[1])

history_fs = fs_dnn.fit(
    X_train_fs, y_train,
    epochs=30,
    batch_size=32,
    validation_split=0.2,
    verbose=0
)

y_pred_fs = (fs_dnn.predict(X_test_fs) > 0.5).astype(int)
acc_fs = accuracy_score(y_test, y_pred_fs)


# ======================================
# 6. 3️⃣ PCA + DNN
# ======================================
pca = PCA(n_components=0.95)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

pca_dnn = create_dnn(X_train_pca.shape[1])

history_pca = pca_dnn.fit(
    X_train_pca, y_train,
    epochs=30,
    batch_size=32,
    validation_split=0.2,
    verbose=0
)

y_pred_pca = (pca_dnn.predict(X_test_pca) > 0.5).astype(int)
acc_pca = accuracy_score(y_test, y_pred_pca)


# ======================================
# 7. 4️⃣ Training & Validation Plots
# ======================================
plt.figure()
plt.plot(history_simple.history['accuracy'], label='Train Accuracy')
plt.plot(history_simple.history['val_accuracy'], label='Val Accuracy')
plt.title('Simple DNN Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


plt.figure()
plt.plot(history_fs.history['accuracy'], label='Train Accuracy')
plt.plot(history_fs.history['val_accuracy'], label='Val Accuracy')
plt.title('Feature Selection + DNN Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


plt.figure()
plt.plot(history_pca.history['accuracy'], label='Train Accuracy')
plt.plot(history_pca.history['val_accuracy'], label='Val Accuracy')
plt.title('PCA + DNN Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


# ======================================
# 8. 5️⃣ Accuracy Comparison Plot
# ======================================
models = ['Simple DNN', 'Feature Selection + DNN', 'PCA + DNN']
accuracies = [acc_simple, acc_fs, acc_pca]

plt.figure()
plt.bar(models, accuracies)
plt.ylabel('Accuracy')
plt.title('Model Comparison')
plt.ylim(0, 1)
plt.show()


# ======================================
# 9. Print Results
# ======================================
print("Final Test Accuracies:")
print(f"Simple DNN               : {acc_simple:.4f}")
print(f"Feature Selection + DNN  : {acc_fs:.4f}")
print(f"PCA + DNN                : {acc_pca:.4f}")
